In [12]:
import webvtt
import jiwer
import pandas as pd
import json

workflow: transcribe result -> txt (jiwer transformation) -> difftxt

## Amazon Transcribe

In [13]:
aws = webvtt.read('aws-transcribe-result.vtt')

In [14]:
aws_transcribe = []
for caption in aws:
    aws_transcribe.append(caption.text)

In [97]:
sample_transcript=pd.read_csv('sample-transcript.txt', sep="/n", header=None)[0].tolist() 

/var/folders/1m/wb29rdrn08q57rfbksgz4z9h0000gn/T/ipykernel_1050/3453676625.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  sample_transcript=pd.read_csv('sample-transcript.txt', sep="/n", header=None)[0].tolist()


## Google Speech-to-text

In [16]:
gcp_transcribe = []
f = open('gcp-transcribe-result.json')
gcp = json.load(f)
for i in gcp['results']:
    gcp_transcribe.append(i["alternatives"][0]["transcript"])

In [17]:
gcp_transcribe=pd.read_csv('gcp-transcribe-result.txt', sep="/n", header=None)[0].tolist() 

/var/folders/1m/wb29rdrn08q57rfbksgz4z9h0000gn/T/ipykernel_1050/1754411159.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  gcp_transcribe=pd.read_csv('gcp-transcribe-result.txt', sep="/n", header=None)[0].tolist()


## Azure Speech-to-text

In [18]:
azure_transcribe=pd.read_csv('azure-transcribe-result.txt', sep="/n",header=None)[0].tolist() 

/var/folders/1m/wb29rdrn08q57rfbksgz4z9h0000gn/T/ipykernel_1050/3963438028.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  azure_transcribe=pd.read_csv('azure-transcribe-result.txt', sep="/n",header=None)[0].tolist()


## Zoom Transcribe

In [24]:
zoom_transcribe=pd.read_csv('zoom-transcribe.txt', sep="/n",header=None)[0].tolist() 

/var/folders/1m/wb29rdrn08q57rfbksgz4z9h0000gn/T/ipykernel_1050/1613688721.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  zoom_transcribe=pd.read_csv('zoom-transcribe.txt', sep="/n",header=None)[0].tolist()


## Youtube Transcribe

In [118]:
yt_transcribe=pd.read_csv('yt-transcribe-result.txt', sep="/n",header=None)[0].tolist() 

/var/folders/1m/wb29rdrn08q57rfbksgz4z9h0000gn/T/ipykernel_1050/940662847.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  yt_transcribe=pd.read_csv('yt-transcribe-result.txt', sep="/n",header=None)[0].tolist()


## WER

In [81]:
transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.ReduceToSingleSentence(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.Strip(),
    jiwer.ExpandCommonEnglishContractions(),
    jiwer.RemovePunctuation(),
    jiwer.ReduceToListOfListOfWords(),
])

In [98]:
from jiwer import wer
aws_error = wer(sample_transcript, aws_transcribe, truth_transform=transformation, hypothesis_transform=transformation)
aws_error

0.12437185929648241

In [99]:
gcp_error = wer(sample_transcript, gcp_transcribe, truth_transform=transformation, hypothesis_transform=transformation)
gcp_error

0.19120603015075377

In [100]:
azure_error = wer(sample_transcript, azure_transcribe, truth_transform=transformation, hypothesis_transform=transformation)
azure_error

0.11758793969849246

In [101]:
zoom_error = wer(sample_transcript, zoom_transcribe, truth_transform=transformation, hypothesis_transform=transformation)
zoom_error

0.23492462311557788

In [119]:
yt_error = wer(sample_transcript, yt_transcribe, truth_transform=transformation, hypothesis_transform=transformation)
yt_error

0.14346733668341707

## Transcription list to text

In [120]:
# #Reference
sample_lst = sum(transformation(sample_transcript), [])
with open("/Users/yooni/Desktop/vca/ref-txt.txt",'w') as f:
    for cap in sample_lst:
        f.write("%s\n" % cap)
        f.write(" ")
    print('Done')

# Aws
aws_lst = sum(transformation(aws_transcribe), [])
with open("/Users/yooni/Desktop/vca/aws-txt.txt",'w') as f:
    for cap in aws_lst:
        f.write("%s\n" % cap)
        f.write(" ")
    print('Done')

# # GCP
gcp_lst = sum(transformation(gcp_transcribe), [])
with open("/Users/yooni/Desktop/vca/gcp-txt.txt",'w') as f:
    for cap in gcp_lst:
        f.write("%s\n" % cap)
        f.write(" ")
    print('Done')

# #Azure
azure_lst = sum(transformation(azure_transcribe), [])
with open("/Users/yooni/Desktop/vca/azure-txt.txt",'w') as f:
    for cap in azure_lst:
        f.write("%s\n" % cap)
        f.write(" ")
    print('Done')

# #Zoom
zoom_lst = sum(transformation(zoom_transcribe), [])
with open("/Users/yooni/Desktop/vca/zoom-txt.txt",'w') as f:
    for cap in zoom_lst:
        f.write("%s\n" % cap)
        f.write(" ")
    print('Done')

# #Youtube
yt_lst = sum(transformation(yt_transcribe), [])
with open("/Users/yooni/Desktop/vca/yt-txt.txt",'w') as f:
    for cap in yt_lst:
        f.write("%s\n" % cap)
        f.write(" ")
    print('Done')

Done
Done
Done
Done
Done
Done


In [113]:
import difflib
awsdiff = []
with open("ref-txt.txt", 'r') as ref:
    with open('aws-txt.txt', 'r') as comp:
        diff = difflib.unified_diff(
            ref.readlines(),
            comp.readlines(),
            fromfile='ref',
            tofile='comp',
        )
        for line in diff:
            for prefix in ('---', '+++', '@@'):
                if line.startswith(prefix):
                    break
            for prefix in ('- ', '+ '):
                if line.startswith(prefix):
                    awsdiff.append(line)

with open("/Users/yooni/Desktop/vca/aws-diff.txt",'w') as f:
    for cap in awsdiff:
        f.write(cap)
    print('Done')

Done


In [117]:
gcpdiff = []
with open("ref-txt.txt", 'r') as ref:
    with open('gcp-txt.txt', 'r') as comp:
        diff = difflib.unified_diff(
            ref.readlines(),
            comp.readlines(),
            fromfile='ref',
            tofile='comp',
        )
        for line in diff:
            for prefix in ('---', '+++', '@@'):
                if line.startswith(prefix):
                    break
            for prefix in ('- ', '+ '):
                if line.startswith(prefix):
                    gcpdiff.append(line)

with open("/Users/yooni/Desktop/vca/gcp-diff.txt",'w') as f:
    for cap in gcpdiff:
        f.write(cap)
    print('Done')

Done


In [115]:
azurediff = []
with open("ref-txt.txt", 'r') as ref:
    with open('azure-txt.txt', 'r') as comp:
        diff = difflib.unified_diff(
            ref.readlines(),
            comp.readlines(),
            fromfile='ref',
            tofile='comp',
        )
        for line in diff:
            for prefix in ('---', '+++', '@@'):
                if line.startswith(prefix):
                    break
            for prefix in ('- ', '+ '):
                if line.startswith(prefix):
                    azurediff.append(line)

with open("/Users/yooni/Desktop/vca/azure-diff.txt",'w') as f:
    for cap in azurediff:
        f.write(cap)
    print('Done')

Done


In [116]:
zoomdiff = []
with open("ref-txt.txt", 'r') as ref:
    with open('zoom-txt.txt', 'r') as comp:
        diff = difflib.unified_diff(
            ref.readlines(),
            comp.readlines(),
            fromfile='ref',
            tofile='comp',
        )
        for line in diff:
            for prefix in ('---', '+++', '@@'):
                if line.startswith(prefix):
                    break
            for prefix in ('- ', '+ '):
                if line.startswith(prefix):
                    zoomdiff.append(line)

with open("/Users/yooni/Desktop/vca/zoom-difftxt",'w') as f:
    for cap in zoomdiff:
        f.write(cap)
    print('Done')

Done


In [121]:
ytdiff = []
with open("ref-txt.txt", 'r') as ref:
    with open('yt-txt.txt', 'r') as comp:
        diff = difflib.unified_diff(
            ref.readlines(),
            comp.readlines(),
            fromfile='ref',
            tofile='comp',
        )
        for line in diff:
            for prefix in ('---', '+++', '@@'):
                if line.startswith(prefix):
                    break
            for prefix in ('- ', '+ '):
                if line.startswith(prefix):
                    ytdiff.append(line)

with open("/Users/yooni/Desktop/vca/yt-difftxt",'w') as f:
    for cap in ytdiff:
        f.write(cap)
    print('Done')

Done
